# AgentCore セルフマネージドメモリ戦略デモ

このノートブックでは、boto3 を使用して Amazon Bedrock AgentCore のセルフマネージドメモリ戦略をセットアップして使用する方法を示します。セルフマネージドメモリ戦略を使用すると、会話イベントによってトリガーされるメモリ抽出と統合のためのカスタムパイプラインを作成できます。

## 仕組み

1. トリガーの設定: 短期記憶イベントに基づいてパイプラインを呼び出すトリガー条件（メッセージ数、アイドルタイムアウト、トークン数）を定義
2. 通知の受信: トリガー条件が満たされると、AgentCore が SNS トピックに通知を発行
3. ペイロードの処理: AgentCore が会話データを S3 バケットに配信
4. メモリレコードの抽出と保存: カスタムパイプラインがペイロードを取得してメモリを処理

セルフマネージドメモリ戦略の詳細については、[公式 AWS ドキュメント](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/memory-self-managed-strategies.html#use-self-managed-strategy)を参照してください。

## セットアップ概要

このデモでは以下を行います:
1. 必要な AWS インフラストラクチャ（S3、SNS、SQS、Lambda、IAM ロール）の作成
2. セルフマネージド戦略を使用した AgentCore メモリの作成
3. メモリ処理パイプラインをデモンストレーションするテストイベントの作成
4. 保存されたメモリの取得と使用をデモンストレーションするエージェントの作成
5. 完了時のリソースのクリーンアップ

## セットアップとインポート

In [ ]:
!pip install -r requirements.txt --quiet

In [ ]:
import boto3
import json
import time
import uuid
import os
from datetime import datetime
from aws_utils import AWSUtils

# Configure AWS region
region_name = 'us-east-1'  # Change to your preferred region
aws_utils = AWSUtils(region_name=region_name)

# Read Lambda function code
with open('lambda_function.py', 'r') as f:
    lambda_code = f.read()


## ステップ 1: ペイロード配信用の S3 バケットの作成

トリガー条件が満たされたときに AgentCore が会話ペイロードを配信する S3 バケットを作成します。

In [ ]:
# Create S3 bucket with a unique name
bucket_name = aws_utils.create_s3_bucket('agentcore-memory-payloads')
print(f"S3 バケットを作成しました: {bucket_name}")

## ステップ 2: メモリジョブ通知用の SNS トピックの作成

AgentCore がメモリ処理パイプラインをトリガーしたときに通知を受信する SNS トピックを作成します。

In [ ]:
# Create SNS topic
sns_topic_name = f"agentcore-memory-notifications-{int(time.time())}"
sns_topic_arn = aws_utils.create_sns_topic(sns_topic_name)
print(f"SNS トピックを作成しました: {sns_topic_arn}")

## ステップ 3: SNS サブスクリプション付き SQS キューの作成

SNS トピックをサブスクライブする SQS キューを作成します。このキューは Lambda 関数をトリガーするメモリジョブ通知を受信します。

In [ ]:
# Create SQS queue and subscribe to SNS topic
queue_name = f"agentcore-memory-queue-{int(time.time())}"
queue_url, queue_arn = aws_utils.create_sqs_queue_with_sns_subscription(queue_name, sns_topic_arn)
print(f"SQS キューを作成しました: {queue_url}")

## ステップ 4: IAM ロールの作成

2 つの IAM ロールを作成します:
1. AgentCore が S3 と SNS にアクセスするためのロール
2. Lambda が S3、SQS、AgentCore API にアクセスするためのロール

In [ ]:
# Create IAM role for AgentCore
agentcore_role_name = f"AgentCoreMemoryExecutionRole-{int(time.time())}"
agentcore_role_arn = aws_utils.create_iam_role_for_agentcore(
    agentcore_role_name, 
    bucket_name, 
    sns_topic_arn
)
print(f"AgentCore IAM ロールを作成しました: {agentcore_role_arn}")

# Create IAM role for Lambda
lambda_role_name = f"LambdaMemoryProcessingRole-{int(time.time())}"
lambda_role_arn = aws_utils.create_iam_role_for_lambda(
    lambda_role_name, 
    bucket_name, 
    queue_arn
)
print(f"Lambda IAM ロールを作成しました: {lambda_role_arn}")

## ステップ 5: メモリ処理用の Lambda 関数の作成

SQS メッセージによってトリガーされる Lambda 関数を作成します。この関数は:
1. S3 から会話ペイロードをダウンロード
2. Bedrock モデルを使用してメモリを抽出
3. 抽出したメモリを AgentCore に保存

In [ ]:
# Create Lambda function
function_name = f"agentcore-memory-processor-{int(time.time())}"
function_arn = aws_utils.create_lambda_function(
    function_name,
    lambda_role_arn,
    lambda_code
)
print(f"Lambda 関数を作成しました: {function_arn}")

# Add SQS trigger to Lambda
event_source_uuid = aws_utils.add_sqs_trigger_to_lambda(function_name, queue_arn)
print(f"Lambda に SQS トリガーを追加しました: {event_source_uuid}")

## ステップ 6: セルフマネージド戦略を使用した AgentCore メモリの作成

セットアップしたインフラストラクチャを使用するセルフマネージド戦略構成で AgentCore メモリを作成します。

In [ ]:

import importlib
import aws_utils
importlib.reload(aws_utils)

# # Create a new instance of AWSUtils with the updated code
aws_utils = aws_utils.AWSUtils(region_name=region_name)

# Create memory with self-managed strategy
memory_name = f"SelfManageMemory{int(time.time())}"
memory_description = "Demo memory using self-managed strategy"

memory_id = aws_utils.create_memory_with_self_managed_strategy(
    memory_name=memory_name,
    memory_description=memory_description,
    role_arn=agentcore_role_arn,
    sns_topic_arn=sns_topic_arn,
    s3_bucket_name=bucket_name,
    message_trigger_count=3,  # Trigger after 3 messages
    token_trigger_count=500,  # Trigger after ~500 tokens
    idle_timeout=300,         # Trigger after 5 minutes of idle time
    historical_window_size=5  # Include 5 previous messages in context
)

print(f"メモリを作成しました: {memory_id}")
# print(f"ストラテジー ID: {strategy_id}")

In [ ]:
def wait_for_memory_to_get_active(memory_id):
    response = aws_utils.agentcore_client_control.get_memory(
        memoryId = memory_id)

    while response['memory']['status'] != 'ACTIVE':
        time.sleep(30)
        response = aws_utils.agentcore_client_control.get_memory(
        memoryId = memory_id)
        print(f"メモリ作成ステータス: {response['memory']['status']}")
    return response['memory']['status']

wait_for_memory_to_get_active(memory_id=memory_id)

## ステップ 7: メモリパイプラインをトリガーするテストイベントの作成

セルフマネージドメモリパイプラインをトリガーするためのテストイベントを作成しましょう。メッセージトリガー数を超えるのに十分なイベントを作成します。

In [ ]:
actor_id = "test-user-123"

In [ ]:
# Create test events
session_id = aws_utils.create_test_events(
    memory_id=memory_id,
    actor_id=actor_id,
    num_events=6  # This will exceed our message_trigger_count of 3
)

print(f"テストイベントを作成しました。セッション ID: {session_id}")

In [ ]:
aws_utils.agentcore_client.list_events(
    memoryId = memory_id, 
    actorId = actor_id,
    sessionId = session_id )

## ステップ 8: メモリ処理の待機

メモリ処理パイプラインが実行されるのを待つ必要があります。これには以下が含まれます:
1. AgentCore がトリガー条件（メッセージ数超過）を検出
2. AgentCore が SNS に通知を発行
3. SNS が SQS にメッセージを配信
4. SQS が Lambda 関数をトリガー
5. Lambda が会話を処理してメモリを保存

少し待ってから、メモリが作成されたかどうかを確認しましょう。

In [ ]:
print("メモリ処理が完了するまで 30 秒待機中...")
time.sleep(30)

## ステップ 9: メモリレコードの確認

メモリパイプラインがメモリレコードを作成したかどうかを、メモリを検索して確認しましょう。

In [ ]:
session_id

In [ ]:
# List memory records
namespace=f"/interests/actor/{actor_id}/session/{session_id}"
def list_memory_records(memory_id, namespace):
    try:
        response = aws_utils.agentcore_client.list_memory_records(
            memoryId=memory_id,
            namespace=namespace
        )
        print(f"{len(response.get('memoryRecordSummaries'))} 件のメモリレコードが見つかりました")
        
        # Display the search results
        for idx, result in enumerate(response.get("memoryRecordSummaries")):
            print(f"メモリ: {idx}")
            print(f"内容: {result['content']['text']}")
    except Exception as e:
        print(f"メモリ検索エラー: {e}")
list_memory_records(memory_id, namespace)

上記のレコードでは、統合ロジックを追加していないためユーザーの興味が繰り返し表示されています。そのため重複がありますが、セルフマネージド戦略を提供する機能により、抽出と取り込みのみを行うかどうかを定義できます。これはビジネスユースケースによって異なります。

In [ ]:
# Search memory records
def retrieve_memory_records(memory_id, query, topK, namespace):
    try:
        response = aws_utils.agentcore_client.retrieve_memory_records(
            memoryId=memory_id,
            searchCriteria = {
            'searchQuery': query,
            'topK': topK
        },
            namespace=namespace
        )
        print(f"{len(response.get('memoryRecordSummaries'))} 件のメモリレコードが見つかりました")
        
        # Display the search results
        for idx, result in enumerate(response.get('memoryRecordSummaries')):
            print(f"\nメモリレコード {idx + 1}:")
            print(f"内容: {result['content']['text']}")
    except Exception as e:
        print(f"メモリ検索エラー: {e}")

retrieve_memory_records(memory_id=memory_id, query="food choices for dinner", topK=5, namespace=namespace)

## ステップ 10: 異なるコンテンツで追加のテストイベントを作成

別のメモリ処理サイクルをトリガーするために、異なるコンテンツでいくつかのテストイベントを作成しましょう。

In [ ]:
# Create custom test events
session_id = str(uuid.uuid4())
actor_id = "test-user-456"

# Custom events with more specific information
test_events = [
    {
        "user": "I'm trying to eat healthier and have been exploring Mediterranean cuisine lately.",
        "assistant": "That's wonderful! Mediterranean food is both delicious and nutritious. What Mediterranean dishes have you tried so far?"
    },
    {
        "user": "I love Greek salads with feta cheese and olives, and I've been making homemade hummus.",
        "assistant": "Homemade hummus is fantastic! Do you prefer it with tahini or without? And what's your favorite way to serve it?"
    },
    {
        "user": "I always use tahini and like to serve it with fresh vegetables and pita bread. I'm also vegetarian, so I avoid meat.",
        "assistant": "Being vegetarian opens up so many Mediterranean options! Have you tried making stuffed grape leaves or lentil-based dishes?"
    },
    {
        "user": "Not yet, but I'd love to learn. I'm also allergic to shellfish, so I have to be careful with seafood dishes.",
        "assistant": "Good to know about the shellfish allergy. For vegetarian Mediterranean cooking, you might enjoy making moussaka with eggplant or trying some traditional Greek bean dishes. Would you like some recipe suggestions?"
    }
]

# Create events
for idx, event in enumerate(test_events):
    try:
        event_payload = [
            {
                'conversational': {
                    'content': {
                        'text': event['user']
                    },
                    'role': 'USER'
                }
            },
            {
                'conversational': {
                    'content': {
                        'text': event['assistant']
                    },
                    'role': 'ASSISTANT'
                }
            }
        ]

        aws_utils.agentcore_client.create_event(
            memoryId=memory_id,
            actorId=actor_id,
            sessionId=session_id,
            eventTimestamp=int(time.time()),
            payload=event_payload,
            clientToken=str(uuid.uuid4())
        )

        print(f"イベント {idx+1}/{len(test_events)} を作成しました")
        time.sleep(1)

    except Exception as e:
        print(f"テストイベント作成エラー: {e}")

print("\nメモリ処理が完了するまで 30 秒待機中...")
time.sleep(30)

## ステップ 11: 新しいメモリの検索

ハイキングとユーザーの犬に関連する新しいメモリを検索しましょう。

In [ ]:
# Search memory records for outdoor activities
namespace=f"/interests/actor/{actor_id}/session/{session_id}"
retrieve_memory_records(memory_id=memory_id, query="dog pets golden retriever", topK=5, namespace=namespace)

## ステップ 12: エージェントの作成

このセクションでは、フックを介して AgentCore セルフマネージドメモリと統合された Strands エージェントを使用して、インテリジェントな料理アシスタントを構築する方法を説明します。ユーザーの食の好み、食事制限、食事履歴の長期記憶に焦点を当て、以前の会話と個人の好みに基づいてパーソナライズされたレストラン推奨を提供します。

In [ ]:
import logging
import json
from typing import Dict
from datetime import datetime
from botocore.exceptions import ClientError

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("customer-support")

# Import required modules
from strands import Agent, tool
from strands.hooks import AfterInvocationEvent, HookProvider, HookRegistry, MessageAddedEvent
from ddgs import DDGS
from bedrock_agentcore.memory import MemoryClient

# Initialize MemoryClient
client = MemoryClient(region_name=region_name)

## ステップ 13: セルフマネージドメモリを使用した料理アシスタント用のメモリフックプロバイダーの作成

フックは、エージェントの実行ライフサイクルの特定のポイントで実行される特別な関数です。カスタムフックプロバイダーはセルフマネージドメモリ戦略を活用して、以下の方法で料理コンテキストを自動的に管理します:

- セルフマネージドメモリレコードから**関連する食の好みを取得**
- 食事制限、料理の好み、食事履歴に関する**コンテキスト情報を新しいクエリに注入**
- バッチ操作を使用して将来の参照のために**食事のやり取りを保存**

これにより、以下のようなシームレスなメモリ体験が実現されます:
- 各クエリを処理する前に保存された食の好みを自動的に取得
- 食事履歴に基づいたコンテキスト対応のレストラン推奨を提供

セルフマネージドアプローチにより、食の好みがどのように保存、取得、使用されて食事推奨体験を向上させるかを完全に制御できます。

In [ ]:
# Helper function to get namespaces from memory strategies list
def get_namespaces(mem_client: MemoryClient, memory_id: str) -> Dict:
    """Get namespace mapping for memory strategies."""
    strategies = mem_client.get_memory_strategies(memory_id)
    return {i["type"]: i["namespaces"][0] for i in strategies}

In [ ]:
class CulinaryAssistantMemoryHooks(HookProvider):
    """Memory hooks for culinary assistant agent"""
    
    def __init__(self, memory_id: str, namespace: str):
        self.memory_id = memory_id
        self.namespace = namespace
    
    def retrieve_food_preferences(self, event: MessageAddedEvent):
        """Retrieve user food preferences before processing dining query"""
        messages = event.agent.messages
        if messages[-1]["role"] == "user" and "toolResult" not in messages[-1]["content"][0]:
            user_query = messages[-1]["content"][0]["text"]
            
            try:
                # Retrieve food preferences using direct API
                response = aws_utils.agentcore_client.retrieve_memory_records(
                    memoryId=self.memory_id,
                    searchCriteria={
                        'searchQuery': user_query,
                        'topK': 5
                    },
                    namespace=self.namespace
                )
                
                memory_records = response.get('memoryRecordSummaries', [])
                
                if memory_records:
                    # Format retrieved preferences
                    preferences_context = []
                    for record in memory_records:
                        content = record.get('content', {}).get('text', '').strip()
                        if content:
                            preferences_context.append(content)
                    
                    # Inject food preferences into the query
                    if preferences_context:
                        context_text = "\n".join(preferences_context)
                        original_text = messages[-1]["content"][0]["text"]
                        messages[-1]["content"][0]["text"] = (
                            f"User Food Preferences:\n{context_text}\n\n{original_text}"
                        )
                        logger.info(f"{len(preferences_context)} 件の食の好みレコードを取得しました")
                
            except Exception as e:
                logger.error(f"食の好み取得に失敗しました: {e}")
    
    def save_dining_interaction(self, event: AfterInvocationEvent):
        """Save dining recommendation interaction after agent response"""
        try:
            messages = event.agent.messages
            if len(messages) >= 2 and messages[-1]["role"] == "assistant":
                # Get last user query and agent response
                user_query = None
                agent_response = None
                
                for msg in reversed(messages):
                    if msg["role"] == "assistant" and not agent_response:
                        agent_response = msg["content"][0]["text"]
                    elif msg["role"] == "user" and not user_query and "toolResult" not in msg["content"][0]:
                        user_query = msg["content"][0]["text"]
                        break
                
                if user_query and agent_response:
                    # Save the interaction using direct API
                    interaction_content = f"Query: {user_query}\nRecommendation: {agent_response}"
                    
                    # You would use create_memory_record API here
                    # aws_utils.agentcore_client.create_memory_record(...)
                    
                    logger.info("食事インタラクションをメモリに保存しました")
                    
        except Exception as e:
            logger.error(f"食事インタラクション保存に失敗しました: {e}")
    
    def register_hooks(self, registry: HookRegistry) -> None:
        """Register culinary assistant memory hooks"""
        registry.add_callback(MessageAddedEvent, self.retrieve_food_preferences)
        registry.add_callback(AfterInvocationEvent, self.save_dining_interaction)
        logger.info("料理アシスタントメモリフックを登録しました")

## ステップ 14: 料理アシスタントエージェントの作成

In [ ]:
# Create memory hooks for culinary assistant
print(memory_id)
culinary_hooks = CulinaryAssistantMemoryHooks(memory_id, namespace)

# Create culinary assistant agent
culinary_agent = Agent(
    hooks=[culinary_hooks],
    model="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    tools=[],  # Update these tools as needed
    state={"actor_id": actor_id, "session_id": session_id},
    system_prompt="""You are the Culinary Assistant, a sophisticated restaurant recommendation assistant.

PURPOSE:
- Help users discover restaurants based on their preferences
- Remember user preferences throughout the conversation
- Provide personalized dining recommendations

You have access to a Memory tool that enables you to:
- Store user preferences (dietary restrictions, favorite cuisines, budget preferences, etc.)
- Retrieve previously stored information to personalize recommendations"""
)

print("✅ Culinary assistant agent created with memory capabilities")

#### エージェントの準備完了

### 料理アシスタントシナリオのテスト

In [ ]:
response1 = culinary_agent("what are the food choices for Dinner?")
print(f"サポートエージェント: {response1}")

## ステップ 15: リソースのクリーンアップ

不要なコストを避けるために、作成したすべてのリソースをクリーンアップしましょう。

In [ ]:
# Clean up all resources
import importlib
import aws_utils
importlib.reload(aws_utils)

# # Create a new instance of AWSUtils with the updated code
aws_utils = aws_utils.AWSUtils(region_name=region_name)

# # Clean up resources with auto-discovery
aws_utils.cleanup_resources(discover_resources=True)
print("All resources have been cleaned up!")

## まとめ

このノートブックでは、以下のことをデモンストレーションしました:

1. セルフマネージドメモリに必要な AWS インフラストラクチャのセットアップ
2. セルフマネージド戦略を使用した AgentCore メモリの作成
3. メモリ処理のトリガー条件の設定
4. Lambda ベースのメモリ処理パイプラインの実装
5. サンプル会話によるメモリシステムのテスト
6. 抽出されたメモリの検索
7. セルフマネージドメモリをテストするための料理エージェントの作成
8. すべてのリソースのクリーンアップ

セルフマネージドメモリ戦略により、メモリ抽出を完全に制御でき、特定のユースケースに適したカスタムパイプラインを構築できます。